<hr style="border:3px solid coral"></hr>

# Cartesian communicators

<hr style="border:3px solid coral"></hr>


#### 1d case
When we distribute one dimensional data to multiple ranks, we divide up our Cartesian grid so that rank 0 is near the left edge of our domain, and the highest rank is at the right edge.  This is typically done using the rank to compute subintervals of the domain. 

    int N_local = N/nprocs;
    double dw = (b-a)/nprocs;
    
    # Processor rank used to determine subintervals of our 1d grid
    double a_local = a + rank*dw;     
    double b_local = a_local + dw;
    
Because we know how the data is laid out, we know how to communicate boundary condition data between processors.      
   
#### 2d case    
When we consider two dimensional domains, however, it is no longer so straightforward to tile the domain with processors.  Even among a few obvious choices, we would need to decide whether we 

* Tile in the x direction

* Tile in the y direction

* Use some other optimal "space filling" strategy for ensure optimal communication performance between ranks.

MPI has anticipated this problem, and provides routines to facilitate grid communication for higher dimensional grids.  These routines are 

* `MPI_Cart_create` : Create a virtual Cartesian communicator

* `MPI_Cart_coords` : Obtain local coordinates within the communicator

* `MPI_Cart_get` : Get information from Cartesian communicator

These routines allow us to set up a "virtual" network which allows us to communicate using an $(I,J)$ grid of virtual ranks.  Moreover, virtual communicators will allow us to easily handle periodic data. 

Read more about [Virtual topologies](https://hpc-tutorials.llnl.gov/mpi/virtual_topologies/).

<hr style="border:3px solid coral"></hr>

## What is a Cartesian communicator?

<hr style="border:3px solid coral"></hr>

So far, we have been using a default communicator `MPI_WORLD_COMM`.  This object stores information about the number of processors available,  the current rank, and is passed to most other MPI routines involved in data communication.  

The communicator can tell us

* How many ranks are currently being used (e.g. `nprocs')

* The current rank (e.g. `rank'`)





<hr style="border:2px solid coral"></hr>

## Shift data and replace in a 1d array

<hr style="border:2px solid coral"></hr>

In this example, each processor in a 1d array stores a single integer value.  We create a Cartesian communicator and then shift each value to a neighboring processor. 

We use the MPI routine `MPI_Cart_create`, with signature

    int MPI_Cart_create ( MPI_Comm comm_old, int ndims, int *dims, int *periods,
                                int reorder, MPI_Comm *comm_cart )
                                
where

* **comm_old** is the existing communicator (e.g. `MPI_COMM_WORLD`)

* **ndims** Grid dimensions (1,2,3,...)

* **dims** an array determine size of grid in each direction.  **dims** should be an array of dimension `ndims`. 

* **periods**  Periodicity of each direction. 

* **reorder** Reorder ranks (set to 0 or 1)

* **comm_cart** Address of new communicator. 

Once we create a new communicator, we can no longer rely on our own rank layout, but instead must query the communicator to get the rank of neighboring processors.  We will get this neighboring rank information using the `MPI_Cart_shift` routine.

    int MPI_Cart_shift ( MPI_Comm comm, int direction, int displ,
                               int *source, int *dest )
                               
* **comm** The newly created communicator

* **direction** The direction to shirt (+1, -1)

* **displ** Displacement value

* **source** Source rank (an output value)

* **dest** Destination rank (an output value)


In the example below, we  shift an integer value to the left or right in our original array. 

* A "displacement" value determines whether we shift right or left. 

    * If **displacement** is 1, each processor sends its value to its right neighbor

    * If **displacement** is -1, each processor sends its value to its left neighbor

If the domain is **periodic**, each processor has a neighbor.  If the domain is not periodic, then the left most processor does not have a left neighbor, and the rightmost processor does not have a right neighbor. 

We also use `MPI_Sendrecv_replace`, which combines a the send buffer and the receive buffer.  The send/receives are coordinated so that the send occurs before the value in the send buffer is replaced with a received value. 

In [1]:
%%file cart_01.c

#include <stdio.h>
#include <stdlib.h>

#include <mpi.h>

enum
{
    LEFT=0,
    RIGHT
};
int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    int root = 0;
    
    // #  Create a new communicator
    MPI_Comm comm_cart;  
    int ndim = 1;
    int dims = nprocs;
    int reorder = 0;        
    int periodicity[1] = {0};
    MPI_Cart_create(MPI_COMM_WORLD, ndim, &dims,  periodicity, reorder, &comm_cart);

    int maxdims = 2;
    int coords;
    MPI_Cart_coords(comm_cart, rank, maxdims, &coords);    
        
    // # Each processor stores a single integer
    int val = coords;  // # will be equal to rank in 1d case.
    
    int data[nprocs];
    MPI_Gather(&val,1,MPI_INTEGER,&data[0],1,MPI_INTEGER,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("Before shift : \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",p);
        printf("\n");
        
        printf("%8s : ","Value");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",data[p]);
        printf("\n");
    }
    
    // # Shift information : 1: receive value from the left; -1 receive from right
    int displ = 1;
        
    int dir = 0;
    int nbr[2];
    MPI_Cart_shift(comm_cart, dir, displ, &nbr[LEFT], &nbr[RIGHT]);
    // # printf("rank : %d; source %d; dest %d\n",rank,nbr[LEFT],nbr[RIGHT]);
    
    int tag = 0;
    // int val_before = val;
    // # Only a send buffer is required, no recv. buffer.
    MPI_Sendrecv_replace(&val, 1, MPI_INTEGER, nbr[RIGHT], tag, 
                         nbr[LEFT], tag, comm_cart, MPI_STATUS_IGNORE);

    if (rank == 0)
    {
        printf("\n");        
        
        if (displ== 1)
            printf("Values are shifted to the right (displ = 1). %d",displ);
        else
            printf("Values are shifted to the left (displ = -1). %d",displ);
        
        if (periodicity[0] == 1)
            printf("Domain is periodic\n\n");
        else
            printf("Domain is not periodic\n\n");
    }
    
    MPI_Gather(&val,1,MPI_INTEGER,data,1,MPI_INTEGER,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("After shift : \n");
        printf("%8s : ","Rank");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",p);
        printf("\n");
        
        printf("%8s : ","Value");
        for(int p = 0; p < nprocs; p++)
            printf("%5d",data[p]);
        printf("\n");
    }

    MPI_Finalize();
}

Overwriting cart_01.c


In [2]:
%%bash 

mpicc -o cart_01 cart_01.c

mpirun -n 8 ./cart_01

Before shift : 
    Rank :     0    1    2    3    4    5    6    7
   Value :     0    1    2    3    4    5    6    7

Values are shifted to the right (displ = 1). 1Domain is not periodic

After shift : 
    Rank :     0    1    2    3    4    5    6    7
   Value :     0    0    1    2    3    4    5    6


#### Question

What happens to values at either end of the array? 

<hr style="border: 2px solid coral"></hr>

# Filling ghost cell values (1d)

<hr style="border: 2px solid coral"></hr>

The Cartesian communicator is particularly useful in filling ghost cells in gridded data from neighboring ranks. 

Below, each processor creates a length 3 array of values $x_{j}$, $j = -1,0,1$.  Each processor computes a value at $x_0$.  

Viewing these values as values of the function $f(x) = x$, each processor computes a value $f(x_0)$.  We can then approximate a derivative $f'(x) = 1$ at $x_0$ using the centered formula

\begin{equation}
x_0 = \frac{x_{1} - x_{-1}}{2h}
\end{equation}

where $x_0$ is local to each rank.  To obtain ghost values at $x_{-1}$ and $x_1$ use `MPI_Sendrecv`.  

To get the source and destination ranks needed for `MPI_Sendrecv`, we create a Cartesian communicator, and use `MPI_Cart_shift` to determine source and destination ranks.

<center>
<table style="margin:20px; border:1px solid black">
<caption>Four processors, each with a single degree of freedom located at cell-centers.       <tr>
        <td><img width=300 style="padding:20px" src="./cart_01.png"></img></td>
        <td><img width=300 style="padding:20px" src="./cart_02.png"></img></td>
    </tr>
</caption>    
</table>    
</center>

To fill all ghost cells, we need to do both a left shift and a right shift. 

At the endpoints, we have to fill in ghost cell values from known values.  To detect that we are at endpoints we can compare a source or destination rank to the value `MPI_PROC_NULL`. 


In [3]:
# Leave blank

In [4]:
%%file cart_02.c

#include <mpi.h>
#include <math.h>

#include <stdio.h>
#include <stdlib.h>

enum
{
    LEFT=0,
    RIGHT
};

enum
{
    DIR_X=0
};

double f(double x)
{
    return x*x/2;
}

int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
    
    int root = 0;
    
    // # Create Cartesian communicator
    MPI_Comm comm_cart;
    int ndim = 1;  // # 1 dimensional communicator (useful for 1d)
    int dims[ndim],periodicity[ndim];
    int reorder = 0;
    
    dims[DIR_X] = nprocs;
    periodicity[DIR_X] = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);
    
    // # Get coordinate of current rank
    int I;
    {
        int maxdims = 2;
        MPI_Cart_coords(comm_cart, rank, maxdims, &I);            
    }
    
    // # construct local array  and index as : q[-1],q[0],q[1]    
    double qmem[3], *q;
    {
        q = &qmem[1];
    }

    // # Width of each processor "domain".
    double a,b,h,x;
    {
        a = 0;     
        b = 1;
        h = (b-a)/dims[DIR_X];
        
        // # Fill in cell centered mesh locations using coordinate I (not rank).
        x = a + (I + 0.5)*h;  // # Value at center of processor domain
        q[0] = f(x);
    }
    
    
    // # Write out data
    int grid_coords[nprocs];
    double x_grid[nprocs]; 
    
    MPI_Gather(&I,1,MPI_INTEGER,grid_coords,1,MPI_INTEGER,root,MPI_COMM_WORLD);
    MPI_Gather(&x,1,MPI_DOUBLE,x_grid,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("(I) (x) coordinates for each rank\n");
        for(int p = 0; p < nprocs; p++)        
            printf("%5d (%d) (%8.4f)\n",p,grid_coords[p],x_grid[p]);
        printf("\n");        
    }        

    int nbr[2];   // # Store left/right neighbors
        
    // #  Fill ghost value at the left : q[-1]
    {
        int dir = DIR_X;  // # only direction 0 in 1d
        int disp = 1;
        int ierr = MPI_Cart_shift(comm_cart, dir, disp, &nbr[LEFT], &nbr[RIGHT]);

        // # Need to check if we have valid source rank from the left
        if (nbr[LEFT] == MPI_PROC_NULL)
            q[-1] = f(x-h);
    
        // # Send values to the right;  receive from the left
        int tag = 0;
        MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, nbr[RIGHT], tag, 
                     &q[-1], 1, MPI_DOUBLE, nbr[LEFT], tag, 
                     comm_cart, MPI_STATUS_IGNORE);        
    }

    // #  Fill ghost value at the right : q[1]
    {
        int dir = DIR_X;
        int disp = -1;  
        MPI_Cart_shift(comm_cart, dir, disp, &nbr[RIGHT], &nbr[LEFT]);

        // # Need to check if we have valid source at the right
        if (nbr[RIGHT] == MPI_PROC_NULL)
            q[1] = f(x+h);
    
        // # Send values to the left;  receive from the right
        int tag = 1;
        int ierr = MPI_Sendrecv(&q[0], 1, MPI_DOUBLE, nbr[LEFT], tag, 
                                &q[1], 1, MPI_DOUBLE, nbr[RIGHT], tag, 
                                comm_cart, MPI_STATUS_IGNORE);        
    }

    // # Compute a derivative using left and right values
    double L = (q[1] - 2*q[0] + q[-1])/(h*h);
    
    // # Write out Laplacian
    double data[nprocs];
    MPI_Gather(&L,1,MPI_DOUBLE,data,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("\n");
        printf("Laplacian of f(x) = 0.5*x^2\n");
        for(int p = 0; p < nprocs; p++)
            printf("%5d %8.4f\n",p,data[p]);
        printf("\n");        
    }
    

    MPI_Finalize();
}

Overwriting cart_02.c


In [5]:
%%bash 

mpicc -o cart_02 cart_02.c

mpirun -n 8 ./cart_02

(I) (x) coordinates for each rank
    0 (0) (  0.0625)
    1 (1) (  0.1875)
    2 (2) (  0.3125)
    3 (3) (  0.4375)
    4 (4) (  0.5625)
    5 (5) (  0.6875)
    6 (6) (  0.8125)
    7 (7) (  0.9375)


Laplacian of f(x) = 0.5*x^2
    0   1.0000
    1   1.0000
    2   1.0000
    3   1.0000
    4   1.0000
    5   1.0000
    6   1.0000
    7   1.0000



<hr style="border: 2px solid coral"></hr>

# Filling ghost cell values (2d)

<hr style="border: 2px solid coral"></hr>

Where the real power of the Cartesian communicator comes in is for higher dimensional grids.  

Below, we alllocate a 2d array, and use a two dimensional communicator. 

To facilitate using finite difference schemes in 2d, we use the same trick as in 1d.  

<center>
<table style="margin:20px; border:1px solid black">
    <caption>Local memory layout for two dimensional grids with ghost cells.</caption>
    <tr>
        <td><img width=400 style="padding-top:20px" src="./cart_03.png"></img></td>
        <td><img width=400 style="padding-top:20px" src="./cart_04.png"></img></td>
    </tr>
</table>    
</center>

This allows us to apply finite difference stencils in two dimensions.   To compute a Laplacian, we can apply the two dimensional finite difference stencil (commonly called the "5-point stencil") as 

\begin{equation}
\nabla^2 u(x) \approx \frac{u_{-1,0} + u_{1,0} + u_{0,-1} + u_{1,1}  - 4 u_{0,0}}{h^2}
\end{equation}

<center>
<table style="margin:20px; border:1px solid black">
    <caption>$4 \times 4$ grid of 16 processors with a single DOF per processor.</caption>
    <tr>
        <td><img width=300 style="padding-top:20px" src="./cart_05.png"></img></td>
        <td><img width=100 style="padding-top:20px" src="./cart_06.png"></img></td>
    </tr>
</table>    
</center>    

Applying this formula to the function $u(x,y) = \frac{1}{4}(x^2 + y^2)$, we have $\nabla^2 u = u_{xx} + u_{yy} = 1$. 

In the code below, we do the following on each processor. 

* Create a $3 \times 3$ grid containing a single degree of freedom surrounded by a buffer layer to hold ghost cell values are values copied from neighboring processors.   These values can be index as 

      q[i][j], i,j = -1,0,1

* Evaluate the function `q[0][0] = f(x)` at unknown locations on each processor. 

* Use a Cartesian communicator to exchange data with neighboring processors

* At the domain boundaries fill ghost cells with know function values.  

* Evaluate the 5-point stencil. 



In [6]:
# Leave blank

In [7]:
%%file cart_03.c

#include <stdio.h>
#include <stdlib.h>

#include <mpi.h>
#include <math.h>

enum
{
    LEFT=0,
    RIGHT,
    DOWN,
    UP
};

enum
{
    DIR_X = 0,
    DIR_Y
};

double f(double x, double y)
{
    return (x*x + y*y)/4.0;
}

int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    int root = 0;
    
    int ndim = 2;
    int dims[ndim];
    if (argc == 3)
    {
        // # Number of processors in each direction.  Product should equal nprocs.
        dims[DIR_X] = atoi(argv[1]);
        dims[DIR_Y] = atoi(argv[2]);
    }
    else
        if (rank == 0)
        {
            printf("Usage : \n");
            printf("cart_03 <dim 0> <dim 1>\n");
            exit(0);            
        }

    if (rank == root)
        if (nprocs != dims[DIR_X]*dims[DIR_Y])
        {
            printf("dim[0]*dim[1] != nprocs\n");
            exit(0);
        }
        
    // # Fill in q[-1] and q[1] with data from neighbors */
    MPI_Comm comm_cart;
    int periodicity[2] = {0,0};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);
    
    // # Create a 3x3 block of memory
    double qmem[9], *qrows[3], **q;
    {
        for(int i = 0; i < 3; i++)
            qrows[i] = &qmem[3*i + 1];
        q = &qrows[1];        
    }
    
    // # Get coordinate of current rank
    int I, J,coords[2];
    {
        int maxdims = 2;
        MPI_Cart_coords(comm_cart,rank,maxdims,coords);

        I = coords[DIR_X];
        J = coords[DIR_Y];                
    }
    
    // # Set grid values
    double hx,hy,x,y;
    {
        double a = 0, b = 1;    // # Assume domain is square.
        hx = (b-a)/dims[DIR_X];   
        hy = (b-a)/dims[DIR_Y];
        x = a + (I+0.5)*hx;
        y = a + (J+0.5)*hy;
    
        // # Value
        q[0][0] = f(x,y);           
    }
    
    // # write out all 2d coordinates.
    int grid_coords[2*nprocs];
    double xy[2] = {x,y};
    double xy_grid[2*nprocs];
    
    MPI_Gather(coords,2,MPI_INTEGER,grid_coords,2,MPI_INTEGER,root,MPI_COMM_WORLD);
    MPI_Gather(xy,2,MPI_DOUBLE,xy_grid,2,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("%8s     (%2s,%2s),   (%8s,%8s)\n","rank","I","J","x","y");
        for(int p = 0; p < nprocs; p++)
        {
            int I = grid_coords[2*p];
            int J = grid_coords[2*p+1];
            double x = xy_grid[2*p];
            double y = xy_grid[2*p + 1];
            printf("%8d     (%2d,%2d),   (%8.4f,%8.4f)\n",p,I,J,x,y);   
        }
        printf("\n");        
    }        

    // # Store source/dest neighbors
    int nbr[4];

    // # Get values at left : q[-1][0]
    {
        int dir = DIR_X;    
        int disp = 1;
        int ierr = MPI_Cart_shift(comm_cart, dir, disp, &nbr[LEFT], &nbr[RIGHT]);

        if (nbr[LEFT] == MPI_PROC_NULL)
            q[-1][0] = f(x-hx,y);
    
        int tag = 0;
        MPI_Sendrecv(&q[0][0], 1, MPI_DOUBLE, nbr[RIGHT], tag, 
                     &q[-1][0], 1, MPI_DOUBLE, nbr[LEFT], tag, 
                     comm_cart, MPI_STATUS_IGNORE);        
    }

    // # Get values at the right : q[1][0]
    {
        int dir = DIR_X;
        int disp = -1;  
        MPI_Cart_shift(comm_cart, dir, disp, &nbr[RIGHT], &nbr[LEFT]);

        if (nbr[RIGHT] == MPI_PROC_NULL)
            q[1][0] = f(x+hx,y);
    
        int tag = 1;
        int ierr = MPI_Sendrecv(&q[0][0], 1, MPI_DOUBLE, nbr[LEFT], tag, 
                                &q[1][0], 1, MPI_DOUBLE, nbr[RIGHT], tag, 
                                comm_cart, MPI_STATUS_IGNORE);        
    }

    // #  Fill value at the bottom : q[0][-1]
    {
        int dir = DIR_Y;    
        int disp = 1;
        int ierr = MPI_Cart_shift(comm_cart, dir, disp, &nbr[DOWN], &nbr[UP]);

        if (nbr[DOWN] == MPI_PROC_NULL)
            q[0][-1] = f(x,y-hy);

        int tag = 2;
        MPI_Sendrecv(&q[0][0], 1, MPI_DOUBLE, nbr[UP], tag, 
                     &q[0][-1], 1, MPI_DOUBLE, nbr[DOWN], tag, 
                     comm_cart, MPI_STATUS_IGNORE);        
    }

    // #  Fill value at the  top : q[0][1]
    {
        int dir = DIR_Y;
        int disp = -1;  
        MPI_Cart_shift(comm_cart, dir, disp, &nbr[UP], &nbr[DOWN]);
    
        if (nbr[UP] == MPI_PROC_NULL)
            q[0][1] = f(x,y+hy);

        int tag = 3;
        int ierr = MPI_Sendrecv(&q[0][0], 1, MPI_DOUBLE, nbr[DOWN], tag, 
                                &q[0][1], 1, MPI_DOUBLE, nbr[UP], tag, 
                                comm_cart, MPI_STATUS_IGNORE);        
    }

    // # Compute a Laplacian at cell center.
    double L = (q[-1][0] + q[1][0] - 2*q[0][0])/(hx*hx) + 
               (q[0][-1] + q[0][1] - 2*q[0][0])/(hy*hy);
    
    // # Write out the Laplacian
    double data[nprocs];
    MPI_Gather(&L,1,MPI_DOUBLE,data,1,MPI_DOUBLE,root,MPI_COMM_WORLD);
    if (rank == root)
    {
        printf("\n");
        printf("Laplacian of f = 0.5*(x^2 + y*2) (cell-centers): \n");
        for(int p = 0; p < nprocs; p++)
            printf("%5d %8.4f\n",p,data[p]);
        printf("\n");        
    }
    MPI_Finalize();
}

Overwriting cart_03.c


In [8]:
%%bash 

mpicc -o cart_03 cart_03.c

# mpirun -n <nprocs> cart_03 <dim[0]> <dim[1]>
# where dim[0]*dim[1] = nprocs

mpirun -n 10 ./cart_03 1 10

    rank     ( I, J),   (       x,       y)
       0     ( 0, 0),   (  0.5000,  0.0500)
       1     ( 0, 1),   (  0.5000,  0.1500)
       2     ( 0, 2),   (  0.5000,  0.2500)
       3     ( 0, 3),   (  0.5000,  0.3500)
       4     ( 0, 4),   (  0.5000,  0.4500)
       5     ( 0, 5),   (  0.5000,  0.5500)
       6     ( 0, 6),   (  0.5000,  0.6500)
       7     ( 0, 7),   (  0.5000,  0.7500)
       8     ( 0, 8),   (  0.5000,  0.8500)
       9     ( 0, 9),   (  0.5000,  0.9500)


Laplacian of f = 0.5*(x^2 + y*2) (cell-centers): 
    0   1.0000
    1   1.0000
    2   1.0000
    3   1.0000
    4   1.0000
    5   1.0000
    6   1.0000
    7   1.0000
    8   1.0000
    9   1.0000



In [88]:
%%file hmwk07_all_2d.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
enum
{
    DIR_X = 0,
    DIR_Y
};

enum
{
    DIRICHLET=0,
    NEUMANN
};

enum
{
    LEFT=0,
    RIGHT,
    BOTTOM,
    TOP
};

int bc_type[4];

double* allocate_1d(int n, int m)
{
    double *mem = (double*) malloc((n + 2*m)*sizeof(double));
    return mem+m;
}

void free_1d(double **x, int m)
{
    free(*x-m);
    *x = NULL;
}

double** allocate_2d(int n, int m, int mbc)
{
    int rows = n + 2*mbc;
    int cols = m + 2*mbc; 

    double   *qmem = malloc(rows*cols*sizeof(double));
    double **qrows = malloc(rows*sizeof(double*));

    for(int i = 0; i < rows; i++)
    {
        qrows[i] = &qmem[cols*i + mbc];
    }    
    return &qrows[mbc];
}

void free_2d(double ***q,int mbc)
{
    free(&(*q)[-mbc][-mbc]);
    free(&(*q)[-mbc]);
    *q = NULL;
}

double u_true(double x,double y)
{
    return sin(2*x)*sin(2*y);
}

double ux_true(double x, double y)
{
    return 2*cos(2*x)*sin(2*y);
}

double uxx_true(double x, double y)
{
    return -4*u_true(x,y);
}

double uy_true(double x, double y)
{
    return 2*sin(2*x)*cos(2*y);
}

double uyy_true(double x, double y)
{
    return -4*u_true(x,y);
}

double un_true(int iside,double x,double y)
{
    double un;
    if (iside == LEFT)
        un = -ux_true(x,y);
    
    else if (iside == RIGHT)
        un = ux_true(x,y);
    
    else if (iside == BOTTOM)
        un = -uy_true(x,y);
    
    else if (iside == TOP)
        un = uy_true(x,y);

    return un;
}

double upp_true(double x,double y)
{
    return uxx_true(x,y) + uyy_true(x,y);
}

Writing hmwk07_all_2d.c


In [194]:
%%file 2d_laplacian.c

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <mpi.h>
#include "hmwk07_all_2d.c"


void comm(int N, double **u, MPI_Comm comm_cart)
{
    int nbr[4];
    double *sendbuf = allocate_1d(N,0);
    double *recvbuf = allocate_1d(N,0);
    
    int tag = 0;

    int source, dest;
    
   // Fill left ghost cell
    {        
        int disp = 1;
        int dir = 0;
        for(int j = 0; j < N; j++)
            sendbuf[j] = u[N-1][j];

        int ierr = MPI_Cart_shift(comm_cart, dir, disp, &nbr[LEFT], &nbr[RIGHT]);

        MPI_Sendrecv(sendbuf, N, MPI_DOUBLE, dest, tag, 
                     recvbuf, N, MPI_DOUBLE, source, tag, 
                     comm_cart, MPI_STATUS_IGNORE);

        if (nbr[LEFT] != MPI_PROC_NULL)
            for(int j = 0; j < N; j++)
                u[-1][j]  = recvbuf[j];       
    }     

   // Fill right ghost cell
    {        
        int disp = -1;
        int dir = 0;
        for(int j = 0; j < N; j++)
            sendbuf[j] = u[0][j];

        int ierr = MPI_Cart_shift(comm_cart, dir, disp, &nbr[RIGHT], &nbr[LEFT]);

        MPI_Sendrecv(sendbuf, N, MPI_DOUBLE, dest, tag, 
                     recvbuf, N, MPI_DOUBLE, source, tag, 
                     comm_cart, MPI_STATUS_IGNORE);


        if (nbr[RIGHT] != MPI_PROC_NULL)
            for(int j = 0; j < N; j++)
                u[N][j]  = recvbuf[j];

    }

  // fill bottom ghost cell
    {
        int disp = 1;
        int dir = 1;
        for(int i = 0; i < N; i++)
            sendbuf[i] = u[i][N-1];

        int ierr = MPI_Cart_shift(comm_cart, dir, disp, &nbr[BOTTOM], &nbr[TOP]);

        MPI_Sendrecv(sendbuf, N, MPI_DOUBLE, dest, tag, 
                     recvbuf, N, MPI_DOUBLE, source, tag, 
                     comm_cart, MPI_STATUS_IGNORE);

        if (nbr[BOTTOM] != MPI_PROC_NULL)
            for(int i = 0; i < N; i++)
                u[i][-1]  = recvbuf[i];
    }

    // Fill top ghost cell
    {        
        int disp = -1;
        int dir = 1;
        for(int i = 0; i < N; i++)
            sendbuf[i] = u[i][0];

        int ierr = MPI_Cart_shift(comm_cart, dir, disp, &nbr[TOP], &nbr[BOTTOM]);

        MPI_Sendrecv(sendbuf, N, MPI_DOUBLE, dest, tag, 
                     recvbuf, N, MPI_DOUBLE, source, tag, 
                     comm_cart, MPI_STATUS_IGNORE);

        if (nbr[TOP] != MPI_PROC_NULL)
            for(int i = 0; i < N; i++)
                u[i][N]  = recvbuf[i];                

    }
    free_1d(&sendbuf,0);
    free_1d(&recvbuf,0);
}


void matvec(int N, double **q, double **L, MPI_Comm comm_cart)
{
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    // # Get current coordinate in comm_cart
    int maxdims = 1;
    int coords[maxdims];
    int periodicity[maxdims];
    int dims[maxdims];        
    
    MPI_Cart_get(comm_cart,maxdims,dims,periodicity,coords);
    int I = coords[DIR_X];    
    int Imax = dims[DIR_X];
    
    int J = coords[DIR_Y];    
    int Jmax = dims[DIR_Y];
    
    int nbr[4];
    
    // # Get values at left : q[-1][j]
    if (I == 0)  
        for(int j = 0; j < N; j++)
        {
          if (bc_type[LEFT] == DIRICHLET)
                q[-1][j] = -q[0][j];
            else
                q[-1][j] = q[0][j];
        }
            
    
    // # Get values at right : q[N][j]
    if (I == Imax-1) 
        for(int j = 0; j < N; j++)
        {
          if (bc_type[RIGHT] == DIRICHLET)
                q[N][j] = -q[N-1][j]; 
            else
                 q[N][j] = q[N-1][j];  
        }
            
        
        
    
    // # Fill value at the bottom and top : q[i][-1], q[i][N]
    if (J == 0)  
        for(int i = 0; i < N; i++)
        {
          if (bc_type[BOTTOM] == DIRICHLET)
                q[i][-1] = -q[i][0];     
            else
               q[i][-1] = q[i][0];  
        }
            
                
    
    if (J == Jmax-1)  
        for(int i = 0; i < N; i++)
        {
          if (bc_type[TOP] == DIRICHLET)
                q[i][N] = -q[i][N-1];     
            else
                q[i][N] = q[i][N-1];  
        }
            
                 
    
    // # Communicate internal boundary data
    comm(N, q, comm_cart);
        
    // # Compute Laplacian (with homogenenous data)
    for(int i = 0; i < N; i++)    
        for(int j = 0; j < N; j++)
            L[i][j] = q[i-1][j] + q[i+1][j] + q[i][j-1] + q[i][j+1] - 4*q[i][j]; 
}
 





int cg(int N, double **F, double **u, double tol, int kmax, int prt,MPI_Comm comm_cart)
{
    // # TODO : Implement at 2d CG solver
    
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    // # Get current coordinate in comm_cart
    int maxdims = 1;
    int coords[maxdims];
    int periodicity[maxdims];
    int dims[maxdims];        
    
    MPI_Cart_get(comm_cart,maxdims,dims,periodicity,coords);
    int I = coords[DIR_X];    
    int Imax = dims[DIR_X];

    int itcount = 0;
    int i1 =0;
    int i2 = N;
    
    double **uk = allocate_2d(N,N,0);
    double **pk = allocate_2d(N,N,1);    
    double **rk = allocate_2d(N,N,0);
    double **Apk = allocate_2d(N,N,0);
    double **rkp1 = allocate_2d(N,N,0);
    
  
    for(int i = i1; i < i2; i++)
     {
      for(int j = i1; j < i2; j++)
    {
        uk[i][j] = 0;
        rk[i][j] = F[i][j];
        pk[i][j] = rk[i][j];    // # Start with uk = 0 --> r = b - Au = b            
    }   
     }   
   
   
 
    for(int k = 0; k < kmax; k++)
    {
        
        matvec(N, pk, Apk, comm_cart);
        
        double rTr = 0;
        double pTAp = 0;
        double alocal[2];
        
        for(int i = i1; i < i2; i++)
        {
            for (int j = i1; j<i2;j++)
            {
            rTr += rk[i][j]*rk[i][j];
            pTAp += pk[i][j]*Apk[i][j]; 
            }
            
        }
        alocal[0] = rTr;
        alocal[1] = pTAp;
        
        double a_1[2];
        MPI_Allreduce(alocal,a_1,2,MPI_DOUBLE,MPI_SUM,MPI_COMM_WORLD);
        
        
        double alpha = rTr/pTAp;
        
        double rpTrp = 0;
        double max_res = 0;
        
        for(int i = i1; i < i2; i++)
        {
            for(int j = i1; j < i2; j++)
            {
                uk[i][j] = uk[i][j] + alpha*pk[i][j];
                rkp1[i][j] = rk[i][j] - alpha*Apk[i][j];
                rpTrp += rkp1[i][j]*rkp1[i][j];
                max_res = fmax(fabs(rkp1[i][j]),max_res);
            }
        }
        
        double b_local;
        
        MPI_Allreduce(&rpTrp,&b_local,1,MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD); 
    
        double error;
        MPI_Allreduce(&max_res,&error,1,MPI_DOUBLE,MPI_MAX,MPI_COMM_WORLD); 
        
        itcount = k+1;
        
        printf("%d error = %12.4e\n",k, max_res); 
        if (error < tol)
            break;
        
        
        double beta = b_local/a_1[0];
        
        for(int i = i1; i < i2; i++)
        {
         for(int j = i1; j < i2; j++)
            {
              pk[i][j] = rkp1[i][j] + beta*pk[i][j];   
            }
                  
        }
        
       
        for(int i = i1; i < i2; i++)
        {
         for (int j =i1; j<i2;j++)
               rk[i][j] = rkp1[i][j];  
        }
           
        rTr = rpTrp;
        
    }
    
    for(int i = i1; i < i2; i++)
    {
      for(int j = i1; j < i2; j++)
        {
          u[i][j] = uk[i][j];    
        }
             
    }
    
    
    
    free_2d(&uk,0);
    free_2d(&pk,1);
    free_2d(&rk,0);
    free_2d(&rkp1,0);
    free_2d(&Apk,0);

    
    
    return itcount;
    
}

int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    int root = 0;
    int N;
    int ndim = 2;
    int dims[ndim];
    if (argc > 3)
    {
        // # Number of processors in each direction.  Product should equal nprocs.
        dims[DIR_X] = atoi(argv[1]);
        dims[DIR_Y] = atoi(argv[2]);
        
    }
    
    if (argc != 7)
    {
        N = 64;
        for(int i = 0; i < 4; i++)
            bc_type[i] = DIRICHLET;
    }        
    else
    {
        N = atoi(argv[3]);    
        for(int i = 0; i < 4; i++)
            bc_type[i] = atoi(argv[4+i]);
    }    

    if (rank == root)
        if (dims[DIR_X]*dims[DIR_Y] !=nprocs)
        {
            printf("dim[0]*dim[1] != nprocs\n");
            exit(0);
        }

    // # Domain
    double a = 0, b = 2*M_PI;
    
    // # Numerical parameters
    double tol = 1e-13;
    int kmax = 10000;    
    int prt = 1;
   
    MPI_Comm comm_cart;
    int periodicity[2] = {0,0};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    // # Get coordinate of current rank
    int I, J, coords[ndim];
    {
        int maxdims = 2;
        MPI_Cart_coords(comm_cart,rank,maxdims,coords);
        I = coords[DIR_X];
        J = coords[DIR_Y];
        
        //printf("Process %d: (%d,%d)\n", rank, coords[0], coords[1]);
    }
    
    // # Create domain  [0,P]x[0,1], where P=nprocs
    int P = nprocs;
   
    // # Set grid values
   
        double dw = (b-a)/P;
        double a_local_x = a + J*dw*dims[DIR_X];
        double b_local_x = a_local_x + dw*dims[DIR_X];
        double a_local_y = a + I*dw*N;
        double b_local_y = a_local_y + dw*N;

        //printf("a_local = %f, b_local = %f\n", a_local_x, b_local_x);

        double h_x = (b_local_x - a_local_x)/dims[DIR_X];
        double h_y = (b_local_y - a_local_y)/dims[DIR_Y];    
    
    
     
    // # Set grid values and a true solution
    int N_local =N/dims[DIR_X];
    // # Arrays    
    double **u = allocate_2d(N_local,N_local,1);
    double **F = allocate_2d(N_local,N_local,0);
    double *x = allocate_1d(N_local,0);
    double *y = allocate_1d(N_local,0);
    
    // # Initialization
    for(int i = 0; i < N_local; i++)
    {
        x[i] = a_local_x + (i+0.5)*h_x;
        for(int j = 0; j < N_local; j++)
        {  
            y[i] = a_local_y + (i+0.5)*h_y;
            F[i][j] = pow(h_x,2)*upp_true(x[i],y[j]);
        }
    }
    
    int Imax = dims[DIR_X];
    int Jmax = dims[DIR_Y];
    
    for(int i = 0; i < N_local; i++)
    {
        if (I == 0)
            {
            if (bc_type[LEFT] == DIRICHLET)
                F[0][i] -=  2* u_true(a_local_x,y[i]);
            else
                F[0][i] -=h_x*un_true(LEFT,a_local_x, y[i]);
            }
        
        if (I == Imax-1)
        {
           if (bc_type[RIGHT] == DIRICHLET)
                F[N_local-1][i] -=2*u_true(b_local_x,y[i]);
            else
                F[N_local-1][i] -=h_x*un_true(RIGHT,b_local_x, y[i]); 
        }
            
        
        if (J == 0)
        {
            if (bc_type[BOTTOM] == DIRICHLET)
                F[i][0] -=2*u_true(a_local_x,x[i]);
            else
                F[i][0] -=h_x*un_true(BOTTOM,a_local_x, x[i]);
            
        }
            
        
        if (J == Jmax-1)
        {
          if (bc_type[TOP] == DIRICHLET)
                F[i][N_local-1] -=2*u_true(b_local_x,x[i]);
            else
                F[i][N_local-1] -=h_x*un_true(TOP,b_local_x, x[i]);   
        }
              
    }
    
    int itcount = cg(N_local,F,u,tol,kmax,prt,comm_cart);
    //printf("itcount = %d\n", itcount);
    
    free_1d(&x,0);
    free_1d(&y,0);
    free_2d(&F,0);
    free_2d(&u,1);
    MPI_Finalize();
    return 0;
}

Overwriting 2d_laplacian.c


In [195]:
%%bash 

mpicc -o 2d_laplacian 2d_laplacian.c

# mpirun -n <nprocs> cart_03 <dim[0]> <dim[1]>
# where dim[0]*dim[1] = nprocs

mpirun -n 16 ./2d_laplacian 4 4 64 

[mpiexec@GRAD-PHDCOMP-02] Sending Ctrl-C to processes as requested
[mpiexec@GRAD-PHDCOMP-02] Press Ctrl-C again to force abort

=   BAD TERMINATION OF ONE OF YOUR APPLICATION PROCESSES
=   PID 47924 RUNNING AT GRAD-PHDCOMP-02
=   EXIT CODE: 2
=   CLEANING UP REMAINING PROCESSES
=   YOU CAN IGNORE THE BELOW CLEANUP MESSAGES
YOUR APPLICATION TERMINATED WITH THE EXIT STRING: Segmentation fault: 11 (signal 11)
This typically refers to a problem with your application.
Please see the FAQ page for debugging suggestions
Process is interrupted.


In [ ]:
void compute_RHS(int N, double **q, double **F, MPI_Comm comm_cart)
{
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    // # Get current coordinate in comm_cart
    int maxdims = 2;
    int coords[maxdims];
    int periodicity[maxdims];
    int dims[maxdims];        
    
    MPI_Cart_get(comm_cart,maxdims,dims,periodicity,coords);
    int I = coords[DIR_X];    
    int J = coords[DIR_Y]; 
    int Imax = dims[DIR_X];
    int Jmax = dims[DIR_Y];
    
    // # Compute right hand side as result of applying five-point Laplacian to 
    // # known solution.
    double **qh = allocate_2d(N,N,1);
    
    for(int i = 0; i < N; i++)
        for(int j = 0; j < N; j++)
            qh[i][j] = q[i][j];
        
     
    
    // # Fix RHS with inhomogeneous data    
    for(int j = 0; j < N; j++)
    {
        if (I == 0)
            F[0][j] +=  (q[-1][j] + q[0][j]);
        
        if (I == Imax-1)
            F[N-1][j] += (q[N-1][j] + q[N][j]); 
        
        if (J == 0) 
            F[j][0] += (q[j][-1] + q[j][0]);
        
        if (J == Jmax-1)
                F[j][N-1] += (q[j][N-1] + q[j][N]);   
    }
    
    
    for(int i = 0; i < N; i++)
    {
        F[i][0] += (q[i][0] + q[i][-1]);
        F[i][N-1] += (q[i][N-1] + q[i][N]);
    }
}

In [13]:
%%file 2d_test.c
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

#define DIM 2

int main(int argc, char **argv) {
    int rank, size;
    int dims[DIM], periods[DIM], coords[DIM];
    int reorder = 0; // No reorder
    MPI_Comm comm_cart;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Define the dimensions of the 2D mesh
    dims[0] = atoi(argv[1]);; // number of rows
    dims[1] = atoi(argv[2]);; // number of columns
    
    if (rank == 0)
        {
            if (dims[0]*dims[1] != size)
            {
               printf("Usage : \n");
                printf("cart_03 <dim 0> <dim 1>\n");
                exit(0); 
            }
                        
        }
    // Set periodic boundary conditions (wraparound)
    periods[0] = 0; // row dimension is periodic
    periods[1] = 0; // column dimension is periodic

    // Create the Cartesian communicator
    MPI_Cart_create(MPI_COMM_WORLD, DIM, dims, periods, reorder, &comm_cart);

    // Get the coordinates of the current process in the 2D mesh
    MPI_Cart_coords(comm_cart, rank, DIM, coords);

    //printf("Process %d: (%d,%d)\n", rank, coords[0], coords[1]);
    
    
    /*
    double b_local_x = a_local_x + dw*dims[DIR_X];
    double a_local_y = a + I*dw*N;
    double b_local_y = a_local_y + dw*N;
    //printf("a_local = %f, b_local = %f\n", a_local_x, b_local_x);
   
    double h_x = (b_local_x - a_local_x)/N;
    double h_y = (b_local_y - a_local_y)/N;
    */
    
    /*
    double dw = (b-a)/P;
    double a_local_x = a + J*dw*dims[DIR_X];
    double b_local_x = a_local_x + dw*dims[DIR_X];
    double a_local_y = a + I*dw*N;
    double b_local_y = a_local_y + dw*N;
    
    //printf("a_local = %f, b_local = %f\n", a_local_x, b_local_x);
   
    double h_x = (b_local_x - a_local_x)/dims[DIR_X];
    double h_y = (b_local_y - a_local_y)/dims[DIR_Y];
    
    */
    
    

    MPI_Finalize();
    return 0;
}


Overwriting 2d_test.c


In [15]:
%%bash 

mpicc -o 2d_test 2d_test.c

# mpirun -n <nprocs> cart_03 <dim[0]> <dim[1]>
# where dim[0]*dim[1] = nprocs

mpirun -n 20 ./2d_test 4 5

Process 0: (0,0)
Process 1: (0,1)
Process 4: (0,4)
Process 6: (1,1)
Process 8: (1,3)
Process 9: (1,4)
Process 11: (2,1)
Process 10: (2,0)
Process 12: (2,2)
Process 16: (3,1)
Process 17: (3,2)
Process 18: (3,3)
Process 19: (3,4)
Process 14: (2,4)
Process 15: (3,0)
Process 13: (2,3)
Process 7: (1,2)
Process 2: (0,2)
Process 3: (0,3)
Process 5: (1,0)


In [ ]:
int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    int root = 0;
    int N;
    int ndim = 2;
    int dims[ndim];
    if (argc > 3)
    {
        // # Number of processors in each direction.  Product should equal nprocs.
        dims[DIR_X] = atoi(argv[1]);
        dims[DIR_Y] = atoi(argv[2]);
        
    }
    

    N = atoi(argv[3]);    
    for(int i = 0; i < 4; i++)
        bc_type[i] = atoi(argv[4+i]);
       

    if (rank == root)
        if (dims[DIR_X]*dims[DIR_Y] !=nprocs)
        {
            printf("dim[0]*dim[1] != nprocs\n");
            exit(0);
        }

    // # Domain
    double a = 0, b = 2*M_PI;
    
    // # Numerical parameters
    double tol = 1e-13;
    int kmax = 10000;    
    int prt = 0;
   
    MPI_Comm comm_cart;
    int periodicity[2] = {0,0};
    int reorder = 0;
    MPI_Cart_create(MPI_COMM_WORLD, ndim, dims,  periodicity, reorder, &comm_cart);

    // # Get coordinate of current rank
    int I, J, coords[ndim];
    {
        int maxdims = 2;
        MPI_Cart_coords(comm_cart,rank,maxdims,coords);
        I = coords[DIR_X];
        J = coords[DIR_Y];
        
    }
    
    // # Create domain  [0,P]x[0,1], where P=nprocs
    int P = nprocs;
   
    // # Set grid values
       
    // Assume Nx = Ny
    int Nx = N/dims[DIR_X];
    
    double dw = (b-a)/dims[DIR_X];
    double a_local_x = a + I*dw;
       
    double a_local_y = a + J*dw;

    double h = (b - a)/N;
            
    
    
     
    // # Set grid values and a true solution
    int N_local =Nx;
    // # Arrays    
    double **u_local = allocate_2d(N_local,N_local,1);
    double **F_local = allocate_2d(N_local,N_local,0);
    double *x_local = allocate_1d(N_local,0);
    double *y_local = allocate_1d(N_local,0);
    
    // # Initialization
    for(int i = 0; i < N_local; i++)
    {
        x_local[i] = a_local_x + (i+0.5)*h;
      
        for(int j = 0; j < N_local; j++)
        {  
            y_local[j] = a_local_y + (j+0.5)*h;
    
            F_local[i][j] = pow(h,2)*upp_true(x_local[i],y_local[j]);
        }
    }
    
    int Imax = dims[DIR_X];
    int Jmax = dims[DIR_Y];
    
    for(int i = 0; i < N_local; i++)
    {
        if (I == 0  )
            {
            if (bc_type[LEFT] == DIRICHLET)
                F_local[0][i] -=  2* u_true(a_local_x,y_local[i]);  
            else
                F_local[0][i] -=h*un_true(LEFT,a, y_local[i]);
                
            }
        
        if (I == Imax-1  )
        {
           if (bc_type[RIGHT] == DIRICHLET)
                F_local[N_local-1][i] -=2*u_true(b,y_local[i]);   
            else
                F_local[N_local-1][i] -=h*un_true(RIGHT,b, y_local[i]); 
        }
            
        
        if (J == 0)
        {
            if (bc_type[BOTTOM] == DIRICHLET)
            {
              F_local[i][0] -=2*u_true(a,x_local[i]); 
            }
                
            else
                F_local[i][0] -=h*un_true(BOTTOM,a,x_local[i]);
            
        }
            
        
        if (J == Jmax-1)
        {
          if (bc_type[TOP] == DIRICHLET)
            {
              F_local[i][N_local-1] -=2*u_true(b,x_local[i]);
                
            }
                
            else
                F_local[i][N_local-1] -=h*un_true(TOP,b,x_local[i]);   
        }
              
    }
    int itcount = cg(N_local,F_local,u_local,tol,kmax,prt,comm_cart);
    if (rank == 0)
        printf("iteration count = %d\n", itcount);
    
      
    double max_err = 0;
       for(int i = 0; i < N_local; i++)
    {
        for(int j = 0; j<N_local; j++)
        {
            max_err = fmax(fabs(u_true(x_local[i],y_local[j]) - u_local[i][j]),max_err);
        }
   
         
    } 
    
    
    double MAXERR;
    MPI_Reduce(&max_err, &MAXERR, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD); 
   
    
    if (rank == 0)
        printf("Error (pde): %12.4e\n", MAXERR);
    
    double *u_old = allocate_1d(N_local*N_local,0);
      for (int j = 0; j<N_local; j++)
    {
        for (int k = 0; k < N_local; k++)
        {
            u_old[N_local*j+k] = u_local[j][k];
            
        }
        
    }
          
    double *x = allocate_1d(N,0);
    double *y = allocate_1d(N,0);
    
      
    if (rank == 0)
    {        
     
         FILE* fout = fopen("err_05.dat","w");   
         fwrite(&max_err,sizeof(double),1,fout);
         fwrite(&itcount,sizeof(int),1,fout);
         fclose(fout);
        
        
        
        double *U = allocate_1d(N*N,0);
        for (int i = 0; i < N_local*N_local; i++)
        {
            U[i] = u_old[i];
        }
        for (int i = 1; i < nprocs; i++) 
        {  
            MPI_Recv(&U[i * N_local*N_local], N_local*N_local, MPI_DOUBLE, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        }
        FILE *file = fopen("hm4_04.dat","wb");      
        fwrite(&N,sizeof(int),1,file);
        fwrite(bc_type,sizeof(int),4,file);
        fwrite(&a,sizeof(double),1,file);
        fwrite(&b,sizeof(double),1,file);
        fwrite(U,sizeof(double),N*N, file); 
    
        fclose(file);   
        free_1d(&U,0);
         
    }
    else
    {
            MPI_Send(u_old, N_local*N_local, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD);
    }
    
    
    free_1d(&x,0);
    free_1d(&y,0);
    free_1d(&u_old,0);    
    free_1d(&x_local,0);
    free_1d(&y_local,0);
    free_2d(&F_local,0);
    free_2d(&u_local,1);
    
    MPI_Finalize();
    return 0;
}